**MOTION PREDICTION OF NEO  
AUTHOR: CHRISTOS DELIGKARIS  
DATE: FEBRUARY 2017**

IMPORT PACKAGES

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import vpython as vp

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

DEFINE THE SCENES

In [2]:
scene3 = vp.canvas(title='VECTORS', x=700, y=0, width=350, height=350, center=vp.vector(5,0,0), background=vp.vector(1,1,1),autoscale=True)
scene2 = vp.canvas(title='WITH RESPECT TO EARTH', x=350, y=0, width=350, height=350, center=vp.vector(5,0,0), background=vp.vector(1,1,1),autoscale=True)
scene1 = vp.canvas(title='WITH RESPECT TO SUN', x=0, y=0, width=350, height=350, center=vp.vector(5,0,0), background=vp.vector(1,1,1))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

CONSTANTS, PARAMETERS

In [3]:
day=90 #day after Jan.1 2006 when prediction begins
deltat_h=86400 #delta t of Horizons data
deltat=864 #delta t used in numerical calculations

In [4]:
object_list=["NEO","Earth","Moon","Sun"]

In [ ]:
#NEO arrays
pos={}
vel={}
#Earth arrays
posEarth={}
velEarth={}
#Moon arrays
posMoon={}
velMoon={}

In [5]:
scene1.select()
Sun=vp.sphere(pos=vp.vector(0,0,0),radius=6.4e6,color=vp.color.orange)

GET POSITIONS AND VELOCITIES OF NEO

In [6]:
def get_horizons_data(file):
    input=open(file,'r')    #open the file with the Horizons system data
    L=input.readlines() #read the file
    i=0 #i will represent the line number (0 for the first line, 1 for the second line, etc)
    imax=len(L) #imax-1 is the number of lines in the input file
    pos={}  #an empty list for positions
    vel={}  #an empty list for velocities
    t=0
    #deltat=86400

    while i<imax:
        if L[i] == "$$SOE\n":   #when we reach the beginning of the data
            i=i+1
            while 1:
                if L[i] == "$$EOE\n": break #stop reading positions and velocities if end of data is reached
                position = L[i+1].split()   #store the position of the object (need to split the string)
                pos[t] = vp.vector(float(position[0]), float(position[1]), float(position[2]))     #convert the strings to floats and then vector
                pos[t] = 1000.*pos[t]   #convert to meters
                velocity = L[i+2].split()   #store the velocity of the object (need to split the string)
                vel[t] = vp.vector(float(velocity[0]), float(velocity[1]), float(velocity[2]))     #convert the strings to floats and then vector
                vel[t] = 1000.*vel[t]   #convert to m/s
                i=i+3       #move to the next position and velocity of the object
                t=t+deltat_h
        i=i+1
    tmax=t-deltat
    return pos,vel,tmax

In [7]:
pos,vel,tmax=get_horizons_data('horizons-results-OtherMoon-wrt-Sun.txt')

In [9]:
t=0
SecondMoonObs = vp.sphere(radius=1.75e6, color=vp.color.black, make_trail=True, pos=pos[t])
while t<=tmax:
    vp.rate(20)
    SecondMoonObs.pos=pos[t]
    t=t+deltat_h

In [8]:
craft=vp.sphere(radius=2e6,color=vp.color.red, make_trail=True, pos=pos[day*deltat_h])

GET POSITIONS AND VELOCITIES OF MOON

In [10]:
posMoon,velMoon,tmaxMoon = get_horizons_data('horizons-results-Moon-wrt-Sun.txt')

In [11]:
t=0
MoonObs = vp.sphere(radius=1.75e6, color=vp.color.red, make_trail=True, pos=posMoon[t])
while t<=tmax:
    #vp.rate(20)
    MoonObs.pos=posMoon[t]
    t=t+deltat_h

In [12]:
Moon=vp.sphere(radius=2*1.75e6,color=vp.color.green, pos=posMoon[day*deltat_h])

GET POSITIONS AND VELOCITIES OF EARTH

In [13]:
posEarth,velEarth,tmaxEarth = get_horizons_data('horizons-results-Earth-wrt-Sun.txt')

In [14]:
t=0
scene2.select()
craft2Obs=vp.sphere(radius=1e6,color=vp.color.black,make_trail=True, pos=pos[t]-posEarth[t])
scene1.select()
EarthObs = vp.sphere(radius=1.75e6, color=vp.color.cyan, make_trail=True, pos=posEarth[t])
while t<=tmax:
    #vp.rate(20)
    craft2Obs.pos=pos[t]-posEarth[t]
    EarthObs.pos=posEarth[t]
    t=t+deltat_h

In [15]:
Earth=vp.sphere(radius=6.4e6,color=vp.color.blue,pos=posEarth[day*deltat_h])
scene2.select()
craft2=vp.sphere(radius=1e6,color=vp.color.red,make_trail=True, pos=craft.pos-Earth.pos)

CREATE FORCE, MOMENTUM ARROWS

In [16]:
scene3.select()
parr_c=vp.arrow(color=vp.color.blue)
fnetarr_c=vp.arrow(color=vp.color.red)#,shaftwidth=2e6)
fnetpararr_c=vp.arrow(color=vp.color.red)
fnetperarr_c=vp.arrow(color=vp.color.red)
fearr_c=vp.arrow(color=vp.color.cyan)#,shaftwidth=2e6)
fmarr_c=vp.arrow(color=vp.color.white)#,shaftwidth=2e6)
fsunarr_c=vp.arrow(color=vp.color.green)
fscale=1e8
pscale=1e1
scene1.select()

GRAPHS

In [17]:
eg = vp.graph(title='ERROR',xtitle='Time [Seconds]',ytitle='Error [meters]',x=0,y=350,width=350,height=350)
ErrorGraph = vp.gcurve(color=vp.color.red)

fng = vp.graph(title='MAGNITUDES OF F NET COMPONENTS', xtitle='Time [Seconds]',ytitle='Force Magnitudes [Newtons]',x=350,y=350,width=350,height=350)
FnetGraph = vp.gcurve(color=vp.color.red)
FnetparGraph = vp.gcurve(color=vp.color.blue)
FnetperGraph = vp.gcurve(color=vp.color.green)

fperg = vp.graph(title='F NET PARALLEL', xtitle='Time [Seconds]',ytitle='F NET PARALLEL [Newtons]',x=700,y=350,width=350,height=350)
FnetparPGraph = vp.gcurve(color=vp.color.blue)
PFnetparGraph = vp.gcurve(color=vp.color.red)

fparg = vp.graph(title='F NET PERPENDICULAR', xtitle='Time [Seconds]',ytitle='F NET PERPENDICULAR [Newtons]',x=1000,y=350,width=350,height=350)
FnetperPGraph = vp.gcurve(color=vp.color.blue)
PFnetperGraph = vp.gcurve(color=vp.color.red)

eng = vp.graph(title='ENERGY-WORK',xtitle='Seconds',ytitle='Joules',x=1000,y=0,width=350,height=350)
EkGraph = vp.gcurve(color=vp.color.magenta)
#Ugraph=gcurve(color=color.yellow)
#Etotal=gcurve(color=color.red)
WGraph = vp.gcurve(color=vp.color.cyan)

enrg = vp.graph(title='ENERGY-WORK',xtitle='Meters',ytitle='Joules',x=1000,y=0,width=350,height=350)
EkrGraph = vp.gcurve(color=vp.color.magenta, dot=True)
EprGraph = vp.gcurve(color=vp.color.green, dot=True)

cg = vp.graph(title='ENERGY-CONSERVATION',xtitle='Seconds',ytitle='Joules',x=1000,y=0,width=350,height=350)
DeltaEGraph = vp.gcurve(color=vp.color.red)

CONSTANTS, PARAMETERS

In [18]:
G=6.67e-11
Earth.m=6e24
craft.m=15e3
Moon.m=7e22
Sun.m=2e30

INITIAL CONDITIONS

In [19]:
craft.v=vel[day*deltat_h]
craft.p=craft.m*craft.v
craft.pinitial = craft.p
craft.pmag=vp.mag(craft.p)
craft.phat = craft.p/craft.pmag
Eki=(vp.mag(craft.pinitial)**2)/(2*craft.m)     #system: craft

Moon.v=velMoon[day*deltat_h]
Moon.p = Moon.m*Moon.v

Earth.v=velEarth[day*deltat_h]
Earth.p=Earth.m*Earth.v

CALCULATE FORCES ON NEO

In [20]:
def get_forces(craftpos,Earthpos,Moonpos,Sunpos):
    
    #calculate forces on NEO
    r_cE=craft.pos-Earth.pos
    rmag_cE=sqrt(r_cE.x**2+r_cE.y**2+r_cE.z**2)  
    rhat_cE=r_cE/rmag_cE
    Fmag_cE=G*craft.m*Earth.m/(rmag_cE**2)
    F_cE=-Fmag_cE*rhat_cE

    r_cM=craft.pos-Moon.pos
    rmag_cM=sqrt(r_cM.x**2+r_cM.y**2+r_cM.z**2)
    rhat_cM=r_cM/rmag_cM
    Fmag_cM=G*craft.m*Moon.m/(rmag_cM**2)
    F_cM=-Fmag_cM*rhat_cM

    r_cS=craft.pos-Sun.pos
    rmag_cS=vp.mag(r_cS)
    rhat_cS=r_cS/rmag_cS
    Fmag_cS=G*craft.m*Sun.m/(rmag_cS**2)
    F_cS=-Fmag_cS*rhat_cS

    Fnet=F_cE + F_cM + F_cS
    
    #calculate forces on MOON
    r_ME = Moon.pos-Earth.pos
    rmag_ME=vp.mag(r_ME)
    rhat_ME=r_ME/rmag_ME
    Fmag_ME = G*Earth.m*Moon.m/(rmag_ME**2)
    F_ME=-Fmag_ME*rhat_ME

    r_Mc=-r_cM
    rmag_Mc=rmag_cM
    rhat_Mc=-rhat_cM
    Fmag_Mc=Fmag_cM
    F_Mc=-F_cM

    r_MS = Moon.pos-Sun.pos
    rmag_MS=vp.mag(r_MS)
    rhat_MS=r_MS/rmag_MS
    Fmag_MS = G*Moon.m*Sun.m/(rmag_MS**2)
    F_MS=-Fmag_MS*rhat_MS

    Fnet_M = F_Mc + F_ME + F_MS
    
    #calculate forces on EARTH
    r_ES = Earth.pos-Sun.pos
    rmag_ES=vp.mag(r_ES)
    rhat_ES=r_ES/rmag_ES
    Fmag_ES = G*Earth.m*Sun.m/(rmag_ES**2)
    F_ES=-Fmag_ES*rhat_ES

    F_EM = - F_ME
    F_Ec = - F_cE

    Fnet_E = F_Ec + F_EM + F_ES
    
    #calculate forces on SUN
    Fnet_S = (-F_ES) + (-F_MS) + (-F_cS)
    
    return Fnet, Fnet_E, Fnet_M, Fnet_S

In [21]:
def check_for_crash(craftpos,Earthpos,Moonpos,Sunpos):
    
    flag=0
    
    r_cE=craft.pos-Earth.pos
    rmag_cE=sqrt(r_cE.x**2+r_cE.y**2+r_cE.z**2)
    if rmag_cE<Earth.radius:
        flag=1
    
    r_cM=craft.pos-Moon.pos
    rmag_cM=sqrt(r_cM.x**2+r_cM.y**2+r_cM.z**2)
    if rmag_cM<Moon.radius:
        flag=2
        
    r_cS=craft.pos-Sun.pos
    rmag_cS=vp.mag(r_cS)
    if rmag_cS<Sun.radius:
        flag=3
        
    return flag

In [22]:
def get_derivatives(coord_and_vel,fnet,m):
    coord=coord_and_vel[0]
    vel=coord_and_vel[1]
    derivatives=np.zeros((2,3))
    for i in range(len(vel)-1):
        derivatives[0,i]=vel[i]
    derivatives[1]=fnet/m    #[0,-g,0]
    return derivatives

In [23]:
def Euler_forw(derivative,f,t,dt):
    deriv=derivative(f,t)
    return np.array([f[0] + (deriv[0]*dt), f[1] + (deriv[1]*dt)])

CALCULATIONS

In [26]:
W_oncraft=0

t=day*deltat_h

while t<1.3*365*24*60*60:
    vp.rate(1e3)
    #scene2.autoscale=False

    craft.phatbefore = craft.phat
    
    ###########################
    #CALCULATE FORCES ON CRAFT

    Fnet,Fnet_E,Fnet_M,Fnet_S = get_forces(craft.pos,Earth.pos,Moon.pos,Sun.pos)
    
    ###########################
    #UPDATE POSITIONS OF ALL OBJECTS
    
    craft_pos_vel = Euler_forw(get_derivatives,[craft.pos,craft.v],Fnet,craft.m)
    craft.pos = np.array([craft_pos_vel[0]])
    craft.v = np.array([craft_pos_vel[1]])
  
    #craft.p=craft.p+Fnet*deltat
    #craft.pos=craft.pos+(craft.p/craft.m)*deltat

    Earth_pos_vel = Euler_forw(get_derivatives,[Earth.pos,Earth.v],Fnet_E,Earth.m)
    Earth.pos = np.array([Earth_pos_vel[0]])
    Earth.v = np.array([Earth_pos_vel[1]])
    
    #Earth.p = Earth.p + Fnet_E*deltat
    #Earth.pos = Earth.pos + (Earth.p/Earth.m)*deltat

    Moon_pos_vel = Euler_forw(get_derivatives,[Moon.pos,Moon.v],Fnet_E,Moon.m)
    Moon.pos = np.array([Moon_pos_vel[0]])
    Moon.v = np.array([Moon_pos_vel[1]])
    
    #Moon.p = Moon.p + Fnet_M*deltat
    #Moon.pos = Moon.pos + (Moon.p/Moon.m)*deltat

    craft2.pos=craft.pos-Earth.pos
    
    ###########################
    #CHECK FOR CRASHES
    
    flag_crash = check_for_crash(craft.pos,Earth.pos,Moon.pos,Sun.pos)
    if(flag_crash != 0):
        print("CRASHED ON ",object_list[flag_crash])

    ###########################
    #CALCULATE F NET COMPONENTS, WORK, ENERGIES

    craft.pmag=vp.mag(craft.p)
    craft.phat=craft.p/craft.pmag
    Fnetpar = vp.dot(Fnet,craft.phat)*craft.phat
    Fnetper = Fnet - Fnetpar

    Ek=(craft.pmag**2)/(2*craft.m)     #system: craft

    #U=-G*Earth.m*craft.m/rmag_cE    #system: craft+earth
    #U=0     #system: craft
    #E=K+U   #system: craft+earth

    deltar=(craft.p/craft.m)*deltat
    W_increment=vp.dot(Fnet, deltar)
    W_oncraft=W_oncraft+W_increment

    #E=K+U+W_oncraft     #system:craft

    #Kgraph.plot(pos=(t,K))
    #this U graph will be compared with the work done (system=craft)
    #Ugraph.plot(pos=(t,U-U0))
    #Ugraph.plot(pos=(t,U))
    #Etotal.plot(pos=(t,E))
    #Wgraph.plot(pos=(t,W_oncraft))

    if abs(t%86400)<10**(-1):
            #print(t,abs(t%86400),mag(craft.pos-pos[round(t)]))
            #ErrorGraph.plot(pos=(t,mag(craft.pos-pos[round(t)])))
            ErrorGraph.plot(pos=(t,vp.mag(craft2.pos-(pos[t]-posEarth[t]))))
    FnetGraph.plot(pos=(t,vp.mag(Fnet)))
    FnetparGraph.plot(pos=(t,vp.mag(Fnetpar)))
    FnetperGraph.plot(pos=(t,vp.mag(Fnetper)))
    FnetparPGraph.plot(pos=(t,1e7*vp.dot(Fnet,craft.phat)))
    PFnetparGraph.plot(pos=(t,craft.pmag))
    FnetperPGraph.plot(pos=(t,vp.mag(Fnetper)))
    #PFnetperGraph.plot(pos=(t,100*math.acos(dot(craft.p,craft.pinitial)/(mag(craft.p)*mag(craft.pinitial)))))
    PFnetperGraph.plot(pos=(t,1e5*math.acos(vp.dot(craft.phat,craft.phatbefore))/deltat))
    EkGraph.plot(pos=(t,Ek))
    WGraph.plot(pos=(t,W_oncraft))
    DeltaEGraph.plot(pos=(t,Ek-Eki-W_oncraft))
    EkrGraph.plot(pos=(vp.mag(craft.pos-Earth.pos),Ek))
    #print(t,100*math.acos(dot(craft.p,craft.pinitial)/(mag(craft.p)*mag(craft.pinitial))))

    #trail.append(pos=craft.pos)
    #parr_c.pos=craft.pos
    parr_c.axis=craft.p*pscale
    #fnetarr_c.pos=craft.pos
    fnetarr_c.axis=fscale*Fnet
    fnetpararr_c.axis = fscale*Fnetpar
    fnetperarr_c.axis = fscale*Fnetper
    #print(Fnet,craft.p)
    #fearr_c.pos=craft.pos
    #fearr_c.axis=fscale*F_cE
    #fmarr_c.pos=craft.pos
    #fmarr_c.axis=fscale*F_cM
    #fsunarr_c.axis=fscale*F_cS

    #scene.center=Sun.pos
    #print(t,1.3*365*24*60*60)
    t=t+deltat

TypeError: get_derivatives() missing 1 required positional argument: 'm'